In [ ]:
import math
import random

# This class represents a node in the Monte Carlo Tree Search (MCTS).
# It stores the game state, its parent, children, the number of visits, and the total value.
class Node:
    def __init__(self, state, parent=None):
        self.state = state
        self.parent = parent
        self.children = []
        self.visits = 0
        self.value = 0

    def is_fully_expanded(self):
        # Checks if all possible actions from the current state have been explored by the children nodes.
        return len(self.children) == len(self.state.get_legal_actions())

    def best_child(self, exploration_weight=1.):
        # Selects the best child node based on the Upper Confidence Bound for Trees (UCT) formula,
        # balancing exploration and exploitation.
        if not self.children:
            return None
        return max(self.children, key=lambda child: child.value / (child.visits + 1) +
                   exploration_weight * math.sqrt(math.log(self.visits + 1) / (child.visits + 1)))

    def expand(self):
        # Expands the current node by creating a new child node for an unexplored action.
        tried_actions = [child.state.last_action for child in self.children]
        legal_actions = self.state.get_legal_actions()
        for action in legal_actions:
            if action not in tried_actions:
                next_state = self.state.move(action)
                child_node = Node(next_state, parent=self)
                self.children.append(child_node)
                return child_node

    def backpropagate(self, result):
        # Updates the node's value and visit count and recursively updates its ancestors to propagate the result of a simulation.
        self.visits += 1
        self.value += result
        if self.parent:
            self.parent.backpropagate(-result)

# This class represents the Tic Tac Toe game, including the board state, current player, and game logic.
class TicTacToe:
    def __init__(self):
        # Initializes the Tic Tac Toe board with an empty state and sets the current player to 'X'.
        self.board = [' '] * 9
        self.current_player = 'X'
        self.last_action = None

    def move(self, index):
        # Returns a new TicTacToe state after making a move at the specified index.
        new_state = TicTacToe()
        new_state.board = self.board[:]
        new_state.board[index] = self.current_player
        new_state.current_player = 'O' if self.current_player == 'X' else 'X'
        new_state.last_action = index
        return new_state

    def get_legal_actions(self):
        # Returns a list of indices representing the legal moves available on the current board.
        return [i for i in range(9) if self.board[i] == ' ']

    def is_terminal(self):
        # Checks if the game has reached a terminal state (either a win or a draw).
        return self.get_winner() is not None or ' ' not in self.board

    def get_winner(self):
        # Returns the winner ('X' or 'O') if there is one, otherwise returns None.
        for (i, j, k) in [(0, 1, 2), (3, 4, 5), (6, 7, 8), (0, 3, 6), (1, 4, 7), (2, 5, 8), (0, 4, 8), (2, 4, 6)]:
            if self.board[i] == self.board[j] == self.board[k] and self.board[i] != ' ':
                return self.board[i]
        return None

    def __str__(self):
        # Returns a string representation of the current board state for displaying it in the console.
        return "\n".join([" | ".join(self.board[i:i + 3]) for i in range(0, 9, 3)])

# Performs Monte Carlo Tree Search (MCTS) to determine the best move from the given initial state.
def mcts(initial_state, itermax=1000, exploration_weight=1.):
    root = Node(initial_state)

    for _ in range(itermax):
        node = root
        while not node.state.is_terminal() and node.is_fully_expanded():
            node = node.best_child(exploration_weight)

        if not node.state.is_terminal():
            node = node.expand()

        result = 1 if node.state.get_winner() == initial_state.current_player else -1 if node.state.get_winner() else 0
        node.backpropagate(result)

    return root.best_child(0).state.last_action

if __name__ == '__main__':
    # Main function to play a game of Tic Tac Toe, where 'X' uses MCTS and 'O' is played by a human.
    state = TicTacToe()
    while not state.is_terminal():
        if state.current_player == 'X':
            action = mcts(state, itermax=1000)
        else:
            while True:
                try:
                    action = int(input("Your move (0-8): "))
                    if action in state.get_legal_actions():
                        break
                    else:
                        print("Invalid move. Try again.")
                except ValueError:
                    print("Please enter a number between 0 and 8.")
        state = state.move(action)
        print(state)
        print("-----")

    winner = state.get_winner()
    if winner:
        print(f"Winner: {winner}")
    else:
        print("It's a draw!")

X |   |  
  |   |  
  |   |  
-----
Your move (0-8): 4
X |   |  
  | O |  
  |   |  
-----
X |   |  
X | O |  
  |   |  
-----
Your move (0-8): 6
X |   |  
X | O |  
O |   |  
-----
X | X |  
X | O |  
O |   |  
-----
Your move (0-8): 2
X | X | O
X | O |  
O |   |  
-----
Winner: O
